## 假设我们想对函数$y=2x^Tx$关于列向量$x$求导

In [10]:
import torch
x = torch.arange(4.0, requires_grad=True)
# 或: x.requires_grad_(True)
x

tensor([0., 1., 2., 3.], requires_grad=True)

## 计算$y$

In [11]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

## 通过调用反向传播函数来自动计算$y$关于$x$每个分量的梯度

In [12]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [13]:
x.grad == 4 * x

tensor([True, True, True, True])

## 现在让我们计算x的另一个函数

In [14]:
# 默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [16]:
x.grad.zero_()
y = x * x  # 此时y是向量，理论上y对x求导结果是一个矩阵

# 在机器学习和深度学习中，我们很少对一个向量的函数进行求导，大部分情况下我们都是对标量进行求导
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 将某些计算移动到记录的计算图之外

In [17]:
x.grad.zero_()
y = x * x
u = y.detach()  # 把y当成常数，而不是关于x的函数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [18]:
x.grad.zero_()
y.sum().backward()  # y仍然是x的函数
x.grad == 2 * x

tensor([True, True, True, True])

## 即使构造函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到变量的梯度

In [20]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)